<a href="https://colab.research.google.com/github/darwinyusef/cognitiveAgent/blob/master/ml/ModeloSentimientosTensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U langchain langchain-openai

In [ ]:
!pip install -U langchain langchain-openai langchain-community

In [ ]:
# @title Datos de entrenamiento básicos
# @markdown <a href="https://github.com/darwinyusef/cognitiveAgent/tree/master">Repositorio con Enlaces a Github</a>
!git clone https://github.com/darwinyusef/cognitiveAgent.git
%cd cognitiveAgent/
!pip install -r requirements.txt

from cognitiveAgent.data.intention_sentences import intention_sentences


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
import numpy as np
import tensorflow as tf
# entrenamiento 2
from sklearn.pipeline import make_pipeline
# entrenamiento 1
from sklearn.preprocessing import LabelEncoder

In [ ]:
from cognitiveAgent.data.sentimental import train_texts_sent

In [ ]:
# @title Modelo de clasificación de sentimientos (Logistic regression multiclass)
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
train_labels_sent = [
    "positivo"] * 25 + ["negativo"] * 25 + ["neutral"] * 19
# En este punto definimos el codificador de etiquetas para iniciar con el entrenamiento  para esto usamos el label encoder
le_sent = LabelEncoder() # -> sckitlearn
y_sent = le_sent.fit_transform(train_labels_sent) # > entran preguntas / etiquetas positivo | negativo | neutral
sentiments = le_sent.classes_.tolist()

# @markdown La arquitectura de embeddings a nivel de sentimientos nos permite convertir y embalar los datos de entrenamiento en embeddings al generar un entrenamiento basado en vectores el entrenamiento permite correr con más datos y en menor tiempo
X_sent = embedding_model.embed_documents(train_texts_sent)
X_sent = np.array(X_sent, dtype=np.float32) # Finalizamos con los embeds y obtenemos una matriz de números

In [ ]:
# @title Agregamos el Modelo simple LRMultiClass
num_classes = len(sentiments)
input_dim = X_sent.shape[1]
#print("imput dim", input_dim, X_sent)

# Configuramos el modelo a enviar de sentimientos
model_sent = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(input_dim,)), #
    tf.keras.layers.Dense(num_classes, activation="softmax")
])

model_sent.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Entrenamiento
model_sent.fit(X_sent, y_sent, epochs=20, batch_size=2, verbose=1)

# En este punto las prdicciones del entrenamiento no se exportan en archivos para agilidad del recurso
def predict_sentiment(text):
    vec = embedding_model.embed_documents([text])
    vec = np.array(vec, dtype=np.float32)
    probs = model_sent.predict(vec) # -> se realiza un proceso de predicción
    idx = np.argmax(probs, axis=1)[0]
    sentiment = le_sent.inverse_transform([idx])[0]
    confidence = float(probs[0, idx])
    return sentiment, confidence


# Funciones de activación matemáticas
![Sofmax](https://c.mql5.com/2/54/all_af.png)

In [ ]:
# @title Codigo para realizar *test*
# fase entrenamiento -> aqui comento varias pruebas para revisar más adelante
for pregunta in intention_sentences[:5]:
    sentimiento, conf = predict_sentiment(pregunta)
    print(f"Pregunta: {pregunta}")
    print(f"→ Sentimiento detectado: {sentimiento} (conf={conf:.2f})\n")

In [ ]:
# @title Finalización del sentimiento y obtención de la predición
# @markdown Este modelo no entrará en la presentación pero queda de apoyo para que vean como intractuan dos modelos de ML dentro de un agente cognitivo

# Ejemplo: analizar solo una sola pregunta <--- solo evalua la var...
sentimiento, conf = predict_sentiment(pregunta)
print(f"Pregunta: {pregunta}")
print(f"→ Sentimiento detectado: {sentimiento} (conf={conf:.2f})")

In [ ]:
import joblib
model_sent.save("/content/train/sentiemiento_tensorflow.h5") # guardar en h5
# Guardar el LabelEncoder en pkl
joblib.dump(le_sent, "/content/train/label_encoder_sentimientos.pkl")

In [ ]:
# @title ML de intensión no será expuesto en la presentación
# Iniciamos otro modelo (Detectar Intensión) este entrenamiento usan otras herramientas más precisas de ml clásico
# pero con un etiquetador especifico en sklearn LinearSVC

train_texts = intention_sentences  # ya es lista de strings
train_labels = ["definicion"] * 4 + ["uso_practico"] * 4 + ["ejemplo"] * 4 + ["sintaxis"] * 4 + \
               ["instalacion"] * 4 + ["depuracion"] * 4 + ["librerias_recomendadas"] * 4 + \
               ["errores_comunes"] * 4 + ["ejercicio_practico"] * 4 + ["concepto_avanzado"] * 4 + \
               ["comparacion"] * 4 + ["recursos_aprendizaje"] * 4 + ["proyecto_guiado"] * 4

# @markdown  - Dentro de los train_text y train_labels encontramos un listado de preguntas x 4 a nivel de concepto y realiza un vs entre la pregunta de la base de conocimiento y el etiquetado generando un entrenamiento sistematico
print(train_texts[12:15])
print(train_labels[12:15])

In [ ]:
# @title Entrenamiento de intensión usando un clasificador con vectores de soporte

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

# lanzamos las frases y los etiquetados
sentences = train_texts[:]
labels = train_labels[:]
# los enviamos al entrenamiento a través de un TfidfVectorizer y los evaluaamos con una Clasificación de Vectores de Soporte
clf = make_pipeline(TfidfVectorizer(), LinearSVC())
clf.fit(sentences, labels)
intencion_predicha = clf.predict([pregunta])[0]
print(intencion_predicha)

In [ ]:
import joblib

# Guardamos el pipeline completo en un archivo
joblib.dump(clf, "/content/train/modelo_intencion.pkl")